# MODULO 5: EVALUACION Y REFINAMIENTO DEL MODELO
### Autor: Guillermo Alfonso Muñiz Hermosillo
### Matricula: A01793101
### Curso: Ciencia y Analitica de Datos
## Profesor: María de La Paz Rico
<h1><center>PROGRESO DEL CURSO:</center></h1>
<img src='./imagenes/A01793101_ProgresoMod5.png'>

<h1><center>GRADED REVIEW QUESTIONS:</center></h1>
<img src='./imagenes/A01793101_GradedReviewQuestionsMod5.png'>

# EVALUACION DEL MODELO

La evaluacion del modelo se refiere a como nuestro modelo se desempeña en el mundo real.

Evaluacion en muestra nos dice como nuestro modelo se acopla a los datos usados para entrenarlo. Pero no nos dice que tan bien nuestro modelo se puede usar para predecir datos nuevos.

Para solucionar esto, dividimos nuestro conjunto de datos en conjunto de muestra o entrenamiento y no de muestra o de prueba.




In [1]:
import pandas as pd
import numpy as np
import ssl
ssl._create_default_https_context = ssl._create_unverified_context


df = pd.read_csv('Cars.csv')

df.head()

,symboling,normalized-losses,make,fuel-type,aspiration,num-of-doors,body-style,drive-wheels,engine-location,wheel-base,...,fuel-system,bore,stroke,compression-ratio,horse-power,peak-rpm,city-mpg,highway-mpg,price,city-L/100km
0,3,122.0,alfa-romero,gas,std,two,convertible,rwd,front,88.6,...,mpfi,3.47,2.68,9.0,111.0,5000,21,27,13495.0,11.190476
1,3,122.0,alfa-romero,gas,std,two,convertible,rwd,front,88.6,...,mpfi,3.47,2.68,9.0,111.0,5000,21,27,16500.0,11.190476
2,1,122.0,alfa-romero,gas,std,two,hatchback,rwd,front,94.5,...,mpfi,2.68,3.47,9.0,154.0,5000,19,26,16500.0,12.368421
3,2,164.0,audi,gas,std,four,sedan,fwd,front,99.8,...,mpfi,3.19,3.40,10.0,102.0,5500,24,30,13950.0,9.791667
4,2,164.0,audi,gas,std,four,sedan,4wd,front,99.4,...,mpfi,3.19,3.40,8.0,115.0,5500,18,22,17450.0,13.055556


Para dividir nuestro conjunto de datos podemos usar la funcion train_test_split de scikit learn


In [18]:
df.dtypes

symboling              int64
normalized-losses    float64
make                  object
fuel-type             object
aspiration            object
num-of-doors          object
body-style            object
drive-wheels          object
engine-location       object
wheel-base           float64
length               float64
width                float64
height               float64
curb-weight            int64
engine-type           object
num-of-cylinders      object
engine-size            int64
fuel-system           object
bore                  object
stroke                object
compression-ratio    float64
horse-power          float64
peak-rpm              object
city-mpg               int64
highway-mpg            int64
price                float64
city-L/100km         float64
dtype: object

In [24]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

onehot = ['make', 'fuel-type', 'aspiration', 'num-of-doors', 'body-style', 'drive-wheels', 'engine-location', 'engine-type', 'num-of-cylinders', 'fuel-system', 'bore', 'stroke', 'peak-rpm']

pipeline_onehot = Pipeline(steps=[
  ('onehot', OneHotEncoder(drop='first', handle_unknown='ignore'))
]);

column_transformer = ColumnTransformer(
    transformers=[
        ('onehot',    pipeline_onehot,                onehot),
    ],
    remainder='passthrough'
)


In [32]:
x_data = df.loc[:, df.columns != 'price']
y_data = df[['price']]

print("X Shape", x_data.shape)
print("y Shape", y_data.shape)

X Shape (201, 26)
y Shape (201, 1)


In [46]:
from sklearn.model_selection import train_test_split



x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.3, random_state=0)

# x_data: Variables independientes 
# y_data: Variable de prediccion o dependiente (Y)
# x_train, y_train: Conjuntos de entrenamiento 
# x_test, y_test: Conjuntos de pruebas
# test_size: Porcentade de datos del conjunto de purebas (30%)
# random_state: Semilla aleatoria para la particion de datos


#x_train = column_transformer.fit_transform(x_train)
#x_test = column_transformer.fit_transform(x_test)
print("X Train", x_train.shape)
print("y Train", y_train.shape)
print("X Test" , x_test.shape)
print("y test" , y_test.shape)

X Train (140, 26)
y Train (140, 1)
X Test (61, 26)
y test (61, 1)


## GENERALIZACION DE DESEMPEÑO

El error de generalizacion es una medida de que tan bien nuestros datos predicen datos no vistos anteriormente
El error que obtenemos usando nuestro conjunto de pruebas es una aproximacion de este error.

## VALIDACION CRUZADA

La validacion cruzada es usada para sobreponerse al error de generalizacion. 
En este metodo el conjunto de datos se parte en grupos de tamaño k, cada grupo se conoce como un "dobles".

Cada grupo, es usado como conjunto de pruebas y validacion repetidamente, al final usamos los resultados promedios para estimar el error de fuera de la muestra.

La metrica de evaluacion de este modelo dependera del modelo

In [38]:
from sklearn.model_selection import cross_val_score
# 1-  Importamos la libreria LinearRegression de Scikit-learn
from sklearn.linear_model import LinearRegression
# 2- Creamos un objeto de regresion lineal usando el constructor
lm = LinearRegression()

x_data_Trans = column_transformer.fit_transform(x_data)
# cv indica el numero de particiones en los conjuntos de datos
scores = cross_val_score(lm, x_data_Trans, y_data, cv=3)

np.mean(scores)

-8.530902040768042

La cuncion cross_val_score nos regresa un marcador para decirnos el resultado de la validacion cruzada

In [43]:
from sklearn.model_selection import cross_val_predict

yhat = cross_val_predict(lm, x_data_Trans, y_data, cv=3)
yhat[0:5]

array([[16303.6559744 ],
       [16303.6559744 ],
       [15201.8872883 ],
       [12809.63219781],
       [15566.8796774 ]])

## SOBREENTRENAMIENTO, SUBENTRENAMIENTO Y SELECCION DEL MODELO

- SUBENTRENAMIENTO: Es aquel modelo que es muy simple para adaptarse a los datos.
- SOBREENTRENAMIENTO: EL modelo se adapta a los datos de entrenamiento pero se desempeña pobremente para estimar la funcion adaptandose mas al ruido.

El error de prueba es una mejor manera de estimar el error en un polinomio. El error disminuye hasta que el mejor polinomio es determinado, despues comienza a incrementar.

In [48]:
from sklearn.preprocessing import PolynomialFeatures

r2Values = []

order = [1,2,3,4,5]

for n in order:
    pr = PolynomialFeatures(degree=n)
    x_train_pr = pr.fit_transform(x_train[['horse-power']])
    x_test_pr = pr.fit_transform(x_test[['horse-power']])
    lm.fit(x_train_pr, y_train)
    r2Values.append(lm.score(x_test_pr, y_test))
    
r2Values

[0.6960381180814083,
 0.697154193256817,
 0.7038917303206413,
 0.7046122605971341,
 0.7403882503151931]

## REGRESION RIDGE

La regression Ridge previene el sobreentrenamiento. Este es un gran problema cuando se tienen multiples variables independientes o caracteristicas.

Esta regresion controla la magnitud de los coeficientes polinomiales al introducir un parametro alpha. 

Alpha es un parametro que seleccionamos antes de entrenar nuestro modelo.

Mientras alpha incrementa, los parametros se vuelven mas pequeños.  Si alpha es muy grande, los coeficientes se acercaran a 0 y se subajustaran a los datos.



In [50]:
from sklearn.linear_model import Ridge

RidgeModel = Ridge(alpha = 0.1)
RidgeModel.fit(x_data_Trans, y_data)
yhatRidge = RidgeModel.predict(x_data_Trans)
yhatRidge[0:5]

array([[13698.75835455],
       [13698.75835455],
       [18810.34830949],
       [10744.43162267],
       [15748.5800395 ]])

## GRID SEARCH
Nos permite escanear a traves de multiples parametros con pocas lineas de codigo. 

Gridsearch toma el modelo u objetos que se requieren entrenar y asigna diferentes valores a los hiperparametros.
Despues calcula el MSE o R2 para estos hiperparametros, permitiendo escoger los mejores valores

In [56]:
from sklearn.model_selection import GridSearchCV

parameters1 = [{
    'alpha': [0.001, 0.1, 1, 10, 100, 1000, 10000, 100000, 10000000],
    }] # Lista de hiperparametros representados en un diccionario.

Grid1 = GridSearchCV(RidgeModel, parameters1, cv=4) # Instanciamos nuestro grid con el modelo, los parametros y cross validation

Grid1.fit(x_data[['horse-power', 'curb-weight', 'engine-size', 'highway-mpg']], y_data) # Hacemos fit a nuestros datos

print(Grid1.best_estimator_) # Obtenemos el mejor parametro

scores =  Grid1.cv_results_ # Obtenemos todos los scores
print(scores['mean_test_score']) # Imprimimos los promedio


Ridge(alpha=10000)
[0.6644635  0.66446408 0.66446932 0.66452138 0.66500981 0.66797719
 0.67241124 0.65695446 0.58042242]
